In [ ]:
haiRaw = sc.textFile("wasb:///hai/HealthcareAssociatedInfections.txt",16)\
    .map(lambda line: [x for x in line.split("\t")])\
    .filter(lambda r: r[0] != 'ProviderID')\
    .map(lambda r: (int(r[0]), str(r[1]), str(r[4]), str(r[8]), str(r[9]), str(r[10]), str(r[11])))
haiRaw.take(1)

In [ ]:
haiSchema = StructType([StructField('ProviderID', IntegerType(), False),
                     StructField('HospitalName', StringType(), True),
                     StructField('State', StringType(), True),
                     StructField('MeasureName', StringType(), True),
                     StructField('MeasureID', StringType(), True),
                     StructField('ComparedToNationalScore', StringType(), True),
                     StructField('MeasureValue', StringType(), True)])
haiDF = sqlContext.createDataFrame(haiRaw, haiSchema)

In [ ]:
haiDF.cache()

In [ ]:
haiDF.show(1)

In [ ]:
haiDF.registerTempTable('hainfections')

In [ ]:
%%sql
SELECT State,
    AVG(MeasureValue) AS AverageScoreMRSA
FROM hainfections
WHERE MeasureID = 'HAI_5_SIR'
GROUP BY State
ORDER BY State

In [ ]:
%%sql
SELECT State,
    SUM(MeasureValue) AS ObservedCasesMRSA
FROM hainfections
WHERE MeasureID = 'HAI_5_NUMERATOR'
GROUP BY State
ORDER BY State

In [ ]:
mrsa = sqlContext.sql("SELECT State, \
                           MeasureValue AS ObservedCasesMRSA \
                       FROM hainfections \
                       WHERE MeasureID = 'HAI_5_NUMERATOR'")
mrsa.select("*").write.save("mrsa.parquet", format="parquet")

In [ ]:
hiveHaiDF = hiveContext.createDataFrame(haiRaw, haiSchema)
hiveHaiDF.saveAsTable('infections')

In [ ]:
%%hive
show tables